1. clases de repuestas cortas con 2 palabras que ocurren más de n veces (k clases)
1.1. contexto de 1 turno anterior
1.1.1. unigramas  (que ocurren más de m veces)
1.1.1.1 unigramas (que ocurren más de 10 veces)
1.1.1.2 unigramas (que ocurren más de 20 veces)
...
1.1.2. unigramas (que ocurren más de m veces) y bigramas (que ocurren más de p veces)
...
1.1.3. unigramas (que ocurren más de m veces) y bigramas (que ocurren más de p veces) y subwords (que ocurren más de r veces)
...
1.1.4. subwords (que ocuren más de r veces)
...
1.2. contexto de 3 turnos anteriores
1.2.1 unigramas...
...
1.3. contexto de 10 turnos anteriores
...

clases de respuestas cortas con 1 palabra que ocurren más de n veces (k clases)
...

clases de respuestas cortas con 1-3 palabras que ocurren más de n veces (k clases)
...

In [2]:
from sklearn.feature_extraction.text import CountVectorizer

/home/jonathan/.virtualenvs/dataMining/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


# Formas de caracterizar en contexto

Haremos los siguientes experimentos para ver la mejor forma de caracterizar el contexto:
    - bolsa de palabras (unigramas, bigramas, subwords, con y sin stopwords)
    - si hay interacción del usuario objetivo y cómo se representa
    - si hay respuestas
    - si hay multimedia y de qué tipo

Usaremos diferentes word embeddings, obtenidos de diferentes tipos de corpus: genéricos o específicos de lenguaje generado por usuario o específicos del corpus del que estamos aprendiendo.
    - Es decir, tres diferentes tipos de word embeddings:
      - SBWCE
      - con el mismo corpus de chats
      - con un corpus de twitter

## Cargando los datos de entrenamiento

In [4]:
import pandas as pd

/home/jonathan/.virtualenvs/dataMining/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [5]:
df = pd.read_csv('short_ans3.csv')

Acordandonos de los datos..

In [6]:
df.head(10)

,idchat,context,label
0,0,NaN,https://m.facebook.com/story.php?story_fbid=14...
1,0,https://m.facebook.com/story.php?story_fbid=14...,nonDef
2,1,😂😂😂😂😂😂😂😂😂😂. Que es este lugar 👀. Qur hace este...,Bienvenido
3,1,Bienvenido. Vengo a pasarle fotos obscenas e i...,👀
4,1,👀. El mundo de nunca jamas?),😂😂
5,1,😂😂,😂😂😂
6,1,😂😂😂,Oh sii
7,1,Oh sii. Seremos niños por siempre,Quienes ? 👀😂
8,1,Quienes ? 👀😂,Yo
9,1,Yo. 😏,😏


## Vectorizando

#### BOG: n-gramas

In [25]:
from string import punctuation

In [26]:
vectorizer = CountVectorizer(min_df=10, stop_words=punctuation)

In [29]:
vectorizer

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=10,
        ngram_range=(1, 1), preprocessor=None,
        stop_words='!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~',
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)